In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import ElementNotVisibleException, ElementNotSelectableException
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from unidecode import unidecode
from urllib.parse import urlparse, parse_qs
from datetime import datetime, timedelta
import re

In [2]:
driver = webdriver.Chrome()

In [3]:
url = "https://www.campings.com/fr/camping/camping-les-salisses-76793#?checkInDate=2024-04-20&night=7"

In [4]:
driver.get(url)

In [5]:
week_scrap = "18/03/2024"

In [6]:
driver.current_url.replace("https://www.campings.com/fr/camping/", '').split('?')

['camping-les-salisses-76793#', 'checkInDate=2024-04-20&night=7']

In [7]:
def get_link_data() -> tuple:
    """ function to get dates in url """
    url = driver.current_url
    data = url.replace("https://www.campings.com/fr/camping/", '').split('?')
    station_key = data[0][:-1]

    try:
        checkin_date = (datetime.strptime(data[1][23:33].replace('-', '/'), "%Y/%m/%d")).strftime("%d/%m/%Y")
    except:
        checkin_date = (datetime.strptime(data[1][12:22].replace('-', '/'), "%Y/%m/%d")).strftime("%d/%m/%Y")

    checkout_date = (datetime.strptime(checkin_date, "%d/%m/%Y") + timedelta(days=7)).strftime("%d/%m/%Y")
    
    return station_key, checkin_date, checkout_date

In [8]:
get_link_data()

('camping-les-salisses-76793', '20/04/2024', '27/04/2024')

In [9]:
soupe = BeautifulSoup(driver.page_source)

In [10]:
results = soupe.find('div', {'class':'results__list'}).find_all('div', {'class':'accommodation-card result__card'})

In [11]:
date = results[0].find('div', {'class':'accommodation-card__offer-dates'}).text.strip().split(',')[0].replace('Du ', '').replace('au ', '')

In [12]:
def extract_dates(string_date,year):
    months = {'janv.': 1, 'févr.': 2, 'mars': 3, 'avr.': 4, 'mai': 5, 'juin': 6, 'juil.': 7, 'août': 8, 'sept.': 9, 'oct.': 10, 'nov.': 11, 'déc.': 12}
    split_date = string_date.split(' ')
    start_date = f"{split_date[0]}/{months[split_date[-1]]}/{year}"
    end_date = f"{split_date[1]}/{months[split_date[-1]]}/{year}"
    return start_date, end_date

In [13]:
extract_dates(date, 2024)

('20/4/2024', '27/4/2024')

In [14]:
data = []
def extract() -> None:
    def extract_dates(string_date,year):
        months = {'janv.': '01', 'févr.': '02', 'mars': '03', 'avr.': '04', 'mai': '05', 'juin': '06', 'juil.': '07', 'août': '08', 'sept.': '09', 'oct.': '10', 'nov.': '11', 'déc.': '12'}
        split_date = string_date.split(' ')
        start_date = f"{split_date[0]}/{months[split_date[-1]]}/{year}"
        end_date = f"{split_date[1]}/{months[split_date[-1]]}/{year}"
        return start_date, end_date

    try:
        soupe = BeautifulSoup(driver.page_source, 'lxml')
        name = ''.join(soupe.find('h1', {'class':'product__name'}).text.strip().split('\n')[:-1]) \
            if soupe.find('h1', class_='product__name') else ''
        localite = ''

        try:
            localite = ''.join(soupe.find('div', class_='product__localisation').text.strip().split('\n')[0].split('-')[1]).replace(", FRANCE", "") \
                if soupe.find('div', class_='product__localisation') else ''
        except IndexError:
            localite = soupe.find('div', class_='product__localisation').text.strip().replace("- Voir sur la carte", "") \
                if soupe.find('div', class_='product__localisation') else ''
        except Exception as e:
            print(e)
            
        try:
            results = soupe.find('div', {'class':'results__list'}).find_all('div', {'class':'accommodation-card result__card'}) \
                if soupe.find('div', {'class':'results__list'}) else []
        except Exception as e:
            print(e)

        datas = []
        station_key, date_debut, date_fin = get_link_data()
        final_results = []

        for result in results:
            dates_string = result.find('div', {'class':'accommodation-card__offer-dates'}).text.strip().split(',')[0].replace('Du ', '').replace('au ', '')
            date_start, date_end = extract_dates(dates_string, year=date_debut.split('/')[2])
            if date_start == date_debut:
                final_results.append(result)

        for result in final_results:
            data = {}
            typologie = result.find('div', {'class':'accommodation-card__name'}).text.strip() \
                if result.find('div', {'class':'accommodation-card__name'}) else ''
            adulte = result.find('div', {'data-property':'adults'}).text.strip() \
                if result.find('div', {'data-property':'adults'}) else ''
            prix_actuel = result.find('div', {'class':'accommodation-card__offer-price'}).text.strip().replace('€', '').replace('\xa0', '').replace(',', '.') \
                if result.find('div', {'class':'accommodation-card__offer-price'}) else ''
            prix_init = result.find('div', {'class':'accommodation-card__offer-old-price'}).find('span').text.strip().replace('€', '').replace('\xa0', '').replace(',', '.') \
                if result.find('div', {'class':'accommodation-card__offer-old-price'}) else prix_actuel

            data['web-scrapper-order'] = ''
            data['date_price'] = week_scrap
            data['date_debut'] = date_debut
            data['date_fin'] = date_fin
            data['prix_init'] = prix_init
            data['prix_actuel'] = prix_actuel
            data['typologie'] = typologie.replace('\n', ' ')
            data['nom'] = name.replace('\n', ' ')
            data['Nb personnes'] = adulte.replace('\n', ' ')
            data['localite'] = localite.replace('\n', ' ')
            data['n_offre'] = ''
            data['date_debut-jour'] = ''
            data['Nb semaines'] = datetime.strptime(date_debut, '%d/%m/%Y').isocalendar()[1]
            data['cle_station'] = station_key
            data['nom_station'] = ''
            data['url'] = driver.current_url.split('?')[-1]
            print(data)
            datas.append(data)

    except Exception as e:
        print(e)
        driver.quit()
        raise Exception(e)


In [ ]:
extract()